# Lifecycle of a Curve

In [1]:
import hvplot.pandas
import pandas as pd

In [63]:
data = pd.Series([1, 2, 3]).to_frame()

hv_obj = data.hvplot(title='A Curve')

hv_obj

:Curve   [index]   (0)

## Tracing the layers of indirection

1. hvPlot: Handles creation of HoloViews (and Panel) objects, translating a simple, flat API into HoloViews objects + Options
2. HoloViews Core: Declaratively expresses the semantics of the data (the Element) + an options specification expressing plot options and style options (including how to map data to color/size/width etc.)
3. HoloViews Plotting: Translates the component and the options into a plotted figure (matplotlib, bokeh, plotly)
4. Panel: Translates the plotted figure into a Bokeh model that can be rendered on the frontend

### 1. hvPlot

Patched accessors on the data objects add the `hvplot` namespace, which in turn invokes the translation layer, i.e. this:

In [64]:
data.hvplot.line(title='A Curve')

:Curve   [index]   (0)

is equivalent to:

In [65]:
from hvplot.plotting import hvPlotTabular

translation_layer = hvPlotTabular(data)

translation_layer.line(title='A Curve')

:Curve   [index]   (0)

Internally the translation layer does a few things:

1. Determine the appropriate `kind` (if not provided) and dispatches to the appropriate method (e.g. `.line` which holds certain defaults)
2. Determine the dimensions and indexes of the data and either infer the requested `x`, `y`, `by`, `groupby` options if needed.
3. Determine if the plot is facetted, e.g. because the user provided `groupby` or `grid` options and then group the dataset appropriately.
4. Create the plot layers, i.e. determine if we are plotting a single curve or multiple curves (e.g. if the user provided `by`) and then reshape the data, e.g. going from wide to tidy data if needed
5. Apply any required operations, e.g. `resample`, `rasterize` or `datashade`.
6. Apply plot and styling to the resulting component.
7. Add any ancilliary components such as tile sources or coastlines.

The result we end up with is a HoloViews component, i.e. and `Element`, `NdOverlay`, `Overlay`, `HoloMap`, `DynamicMap`, `NdLayout` or `GridSpace`.

### 2. HoloViews Core

When hvPlot creates the HoloViews objects a few things happen, at the lowest level of the hierarchy are `Element`s.

#### `Element`

An `Element` is a light wrapper around data, in most cases data can be wrapped without converting it at all but the `DataInterface` layer in HoloViews.

In [69]:
hv_obj.interface

holoviews.core.data.pandas.PandasInterface

The interface does a number of things:

- Validate that the data has the dimension(s) specified OR infer the dimensions based on the dimensionality of the data and the columns/indexes/coordinates of the data.
- Provide an abstraction to compute the types of the data, return the values of the data as arrays and perform certain operations such as computing the range, grouping by some dimension and much more.

In [80]:
hv_obj.interface.values(hv_obj, '0')

array([1, 2, 3])

In [81]:
hv_obj.interface.range(hv_obj, 'index')

(0, 2)

During instantiation the interface is resolved based on the valid datatype(s):

In [89]:
hv.Curve([(0, 1), (0, 1)], datatype=['array', 'dictionary', 'dataframe']).data

array([[0, 1],
       [0, 1]])

In [96]:
hv.Curve.param.vdims.default

[Dimension('y')]

In [99]:
from xarray import load_dataset, tutorial

In [ ]:
tutorial.air

In [82]:
hv_obj.datatype

['dataframe',
 'dictionary',
 'grid',
 'xarray',
 'multitabular',
 'spatialpandas',
 'dask_spatialpandas',
 'dask',
 'cuDF',
 'array',
 'ibis']

Resolution checks if the `data` matches one of the datatypes first and then goes through the datatypes in order to see if they can coerce the data into that format.

#### Options

One of the core design principles of HoloViews was that the semantic information about the data such as what it is and what the units are would be separate from "plotting" or "display" concerns. This is where the options system comes in. Options therefore are not stored on the components themselves but rather on the `Store`:

In [ ]:
hv.Curve([]).opts(width=300, color='red')

In [106]:
import holoviews as hv

hv.Store.options(backend='bokeh')

OptionTree(groups=dict_keys(['style', 'plot', 'norm', 'output']),
   style={
            Area         : dict(alpha=1, color=<Cycle Cycle00229>, line_color=black, muted_alpha=0.2),
            Arrow        : dict(arrow_size=10),
            Bars         : dict(bar_width=0.8, color=<Cycle Cycle00228>, line_color=black, muted_alpha=0.2),
            Bounds       : dict(color=black),
            Box          : dict(color=black),
            BoxWhisker   : dict(box_fill_color=<Cycle Cycle00223>, box_line_color=black, outlier_color=black, whisker_color=black),
            Chord        : dict(edge_hover_line_color=limegreen, edge_line_color=black, edge_line_width=1, edge_nonselection_alpha=0.1, edge_nonselection_line_color=black, edge_selection_line_color=limegreen, label_text_font_size=8pt, node_color=<Cycle Cycle00245>, node_hover_fill_color=limegreen, node_hover_line_color=black, node_line_color=black, node_nonselection_alpha=0.2, node_nonselection_fill_color=<Cycle Cycle00246>, node_nonse

In [112]:
hv.Curve([1, 2, 3], group='stocks')

:Curve   [x]   (y)

The base tree holds the various defaults while applying `.opts()` will create a custom tree. Think of this something like CSS, where the DOM nodes are the elements and the CSS stylesheets are the option trees.

Calling `.opts` will create a new `OptionTree` on the Store, which is resolved together with the default options when the plot is rendered:

In [113]:
hv.Store.custom_options()

{2: OptionTree(groups=dict_keys(['style', 'plot', 'norm', 'output']),
    style={Curve : dict(line_width=2, muted_alpha=0.2)},
 
    plot={
             Curve     : dict(height=300, labelled=[], logx=False, logy=False, responsive=False, shared_axes=True, show_grid=False, show_legend=True, title=A Curve, tools=['hover'], width=700),
             NdOverlay : dict(batched=False, height=300, legend_position=right, logx=False, logy=False, responsive=False, shared_axes=True, show_grid=False, show_legend=True, title=A Curve, tools=['hover'], width=700)},
 
    norm={Curve : dict(axiswise=False, framewise=True)}
 ),
 8: OptionTree(groups=dict_keys(['style', 'plot', 'norm', 'output']),
    style={Curve : dict(line_width=2, muted_alpha=0.2)},
 
    plot={
             Curve     : dict(height=300, labelled=[], logx=False, logy=False, responsive=False, shared_axes=True, show_grid=False, show_legend=True, title=A Curve, tools=['hover'], width=700),
             NdOverlay : dict(batched=False, heigh

### 3. HoloViews Plotting

The plan was to keep the core datastructures of HoloViews entirely separate from the implementation of the plotting backends, so that new backends could be written as extensions to HoloViews. While this vision never fully materialized it is still true. 

The plotting system in HoloViews can be invoked directly, via the `repr` system of notebook environments or via Panel. The entrypoint to the plotting system are the `Renderer` classes, these are 
responsible for taking a HoloViews object and creating a corresponding `Plot`.

#### Renderer

By default a global `Renderer` instance is invoked which has a variety of options on how to render the object:

In [114]:
renderer = hv.renderer('bokeh')

renderer

BokehRenderer(backend='bokeh', center=True, css={}, dpi=None, fig='auto', fps=20, holomap='auto', info_fn=None, key_fn=None, mode='default', name='BokehRenderer00221', post_render_hooks={'svg': [], 'png': []}, size=100, theme=<bokeh.themes.theme.Theme object at 0x16a823550>, webgl=True, widget_location=None, widget_mode='embed')

The usual entrypoint of taking an object and turning it to a plot is the `get_plot` method:

In [122]:
plot = renderer.get_plot(hv_obj)

plot

CurvePlot(active_tools=None, align='start', apply_extents=True, apply_hard_bounds=False, apply_ranges=True, aspect=None, autorange=None, backend_opts={}, bgcolor=None, border=10, data_aspect=None, default_span=2.0, default_tools=['save', 'pan', 'wheel_zoom', 'box_zoom', 'reset'], fontscale=None, fontsize={'title': '12pt'}, frame_height=None, frame_width=None, gridstyle={}, height=300, hooks=[], hover_formatters=None, hover_mode='mouse', hover_tooltips=None, interpolation='linear', invert_axes=False, invert_xaxis=False, invert_yaxis=False, labelled=['y'], lod={'factor': 10, 'interval': 300, 'threshold': 2000, 'timeout': 500}, logx=False, logy=False, margin=None, max_height=None, max_width=None, min_height=None, min_width=None, multi_y=False, name='CurvePlot01365', normalize=True, padding=(0, 0.1), projection=None, responsive=False, scalebar=False, scalebar_label='@{value} @{unit}', scalebar_location='bottom_right', scalebar_opts={}, scalebar_range='x', scalebar_tool=True, scalebar_unit=

The `Renderer` resolves the correct plot type for a given object using the plot registry:

In [124]:
hv.Store.registry['bokeh']

{holoviews.core.overlay.Overlay: holoviews.plotting.bokeh.element.OverlayPlot,
 holoviews.core.overlay.NdOverlay: holoviews.plotting.bokeh.element.OverlayPlot,
 holoviews.core.spaces.GridSpace: holoviews.plotting.bokeh.plot.GridPlot,
 holoviews.core.spaces.GridMatrix: holoviews.plotting.bokeh.plot.GridPlot,
 holoviews.core.layout.AdjointLayout: holoviews.plotting.bokeh.plot.AdjointLayoutPlot,
 holoviews.core.layout.Layout: holoviews.plotting.bokeh.plot.LayoutPlot,
 holoviews.core.layout.NdLayout: holoviews.plotting.bokeh.plot.LayoutPlot,
 holoviews.element.chart.Curve: holoviews.plotting.bokeh.chart.CurvePlot,
 holoviews.element.chart.Bars: holoviews.plotting.bokeh.chart.BarPlot,
 holoviews.element.geom.Points: holoviews.plotting.bokeh.chart.PointPlot,
 holoviews.element.chart.Scatter: holoviews.plotting.bokeh.chart.PointPlot,
 holoviews.element.chart.ErrorBars: holoviews.plotting.bokeh.chart.ErrorPlot,
 holoviews.element.chart.Spread: holoviews.plotting.bokeh.chart.SpreadPlot,
 holovi

Internally the plotting classes corresponding to composite objects such as the `OverlayPlot` will recreate a similarly hierarchy as the components themselves, e.g. and overlay of curves will result in a `OverlayPlot`:

In [125]:
oplot = renderer.get_plot(hv.Curve([]) * hv.Curve([]))

oplot

OverlayPlot(active_tools=None, align='start', apply_extents=True, apply_hard_bounds=False, apply_ranges=True, aspect=None, autorange=None, backend_opts={}, batched=False, bgcolor=None, border=10, data_aspect=None, default_span=2.0, default_tools=['save', 'pan', 'wheel_zoom', 'box_zoom', 'reset'], fontscale=None, fontsize={'title': '12pt'}, frame_height=None, frame_width=None, gridstyle={}, height=300, hooks=[], hover_formatters=None, hover_mode='mouse', hover_tooltips=None, invert_axes=False, invert_xaxis=False, invert_yaxis=False, labelled=['x', 'y'], legend_cols=0, legend_labels=None, legend_limit=25, legend_muted=False, legend_offset=(0, 0), legend_opts={}, legend_position='top_right', lod={'factor': 10, 'interval': 300, 'threshold': 2000, 'timeout': 500}, logx=False, logy=False, margin=None, max_height=None, max_width=None, min_height=None, min_width=None, multi_y=False, multiple_legends=False, name='OverlayPlot01385', normalize=True, padding=0.1, projection=None, responsive=False,

containing individual `CurvePlot` subplots:

In [126]:
oplot.subplots

{('Curve',
  'I'): CurvePlot(active_tools=None, align='start', apply_extents=True, apply_hard_bounds=False, apply_ranges=True, aspect=None, autorange=None, backend_opts={}, bgcolor=None, border=10, data_aspect=None, default_span=2.0, default_tools=['save', 'pan', 'wheel_zoom', 'box_zoom', 'reset'], fontscale=None, fontsize={'title': '12pt'}, frame_height=None, frame_width=None, gridstyle={}, height=300, hooks=[], hover_formatters=None, hover_mode='mouse', hover_tooltips=None, interpolation='linear', invert_axes=False, invert_xaxis=False, invert_yaxis=False, labelled=['x', 'y'], lod={'factor': 10, 'interval': 300, 'threshold': 2000, 'timeout': 500}, logx=False, logy=False, margin=None, max_height=None, max_width=None, min_height=None, min_width=None, multi_y=False, name='CurvePlot01397', normalize=True, padding=(0, 0.1), projection=None, responsive=False, scalebar=False, scalebar_label='@{value} @{unit}', scalebar_location='bottom_right', scalebar_opts={}, scalebar_range='x', scalebar_t

The plots themselves internally do two main things, translate the elements and options into backend specific figures and implement approaches for updating the plots inplace, e.g. if we have a DynamicMap that updates the `Curve`, changes in the data and the options are translated into (ideally minimal updates to the underlying figure object).

In [128]:
hv_obj

:Curve   [index]   (0)

In [132]:
hv.HoloMap({i: hv.Curve([(0, 1), (1, 2), (2, i)]) for i in range(3)})

:HoloMap   [Default]
   :Curve   [x]   (y)

In [131]:
hv.Store._display_hooks

defaultdict(dict,
            {'html+js': {holoviews.core.dimension.LabelledData: <function holoviews.ipython.display_hooks.pprint_display(obj)>},
             'png': {holoviews.core.dimension.LabelledData: <function holoviews.ipython.display_hooks.png_display(element, max_frames)>},
             'svg': {holoviews.core.dimension.LabelledData: <function holoviews.ipython.display_hooks.svg_display(element, max_frames)>}})

In [130]:
hv.Store.render??

Signature: hv.Store.render(obj)
Source:   
    @classmethod
    def render(cls, obj):
        """
        Using any display hooks that have been registered, render the
        object to a dictionary of MIME types and metadata information.
        """
        class_hierarchy = inspect.getmro(type(obj))
        hooks = []
        for _, type_hooks in cls._display_hooks.items():
            for subcls in class_hierarchy:
                if subcls in type_hooks:
                    hooks.append(type_hooks[subcls])
                    break

        data, metadata = {}, {}
        for hook in hooks:
            ret = hook(obj)
            if ret is None:
                continue
            d, md = ret
            data.update(d)
            metadata.update(md)
        return data, metadata
File:      ~/development/holoviews/holoviews/core/options.py
Type:      method

In [129]:
hv_obj._repr_mimebundle_

Signature: hv_obj._repr_mimebundle_(include=None, exclude=None)
Source:   
    def _repr_mimebundle_(self, include=None, exclude=None):
        """
        Resolves the class hierarchy for the class rendering the
        object using any display hooks registered on Store.display
        hooks.  The output of all registered display_hooks is then
        combined and returned.
        """
        return Store.render(self)
File:      ~/development/holoviews/holoviews/core/dimension.py
Type:      method

## 4. Panel

Panel is the final layer and is responsible for taking the underlying figure objects and rendering them in the browser.

How precisely this works is backend dependent but effectively the `HoloViews` pane in Panel dispatches to the appropriate Pane type depending on the backend, specifically:

- The Bokeh backend is rendered with the `Bokeh`
- The Matplotlib backend is rendered with the `Matplotlib` pane.
- The Plotly backend is rendered with the `Plotly` pane.

In [144]:
import panel as pn

hv_pane = pn.pane.HoloViews(hv.HoloMap({i: hv.Curve([(0, 1), (1, 2), (2, i)]) for i in range(3)}))

hv_pane

HoloViews(HoloMap, height=300, sizing_mode='fixed', width=300)

Internally the `HoloViews` pane creates a layout that respects the centered option and optionally renders widgets originating from the `HoloMap` / `DynamicMap` dimensions:

In [138]:
print(hv_pane.layout)

Row
    [0] HoloViews(HoloMap, height=300, sizing_mode='fixed', width=300)
    [1] WidgetBox(align=('end', 'start'))
        [0] DiscreteSlider(formatter='%d', name='Default', options={'0': 0, '1': 1, '2': 2}, value=0)


It then invokes the `renderer` to create a HoloViews `Plot`:

In [145]:
plot, pane = list(hv_pane._plots.values())[0]

plot

CurvePlot(active_tools=None, align='start', apply_extents=True, apply_hard_bounds=False, apply_ranges=True, aspect=None, autorange=None, backend_opts={}, bgcolor=None, border=10, data_aspect=None, default_span=2.0, default_tools=['save', 'pan', 'wheel_zoom', 'box_zoom', 'reset'], fontscale=None, fontsize={'title': '12pt'}, frame_height=None, frame_width=None, gridstyle={}, height=300, hooks=[], hover_formatters=None, hover_mode='mouse', hover_tooltips=None, interpolation='linear', invert_axes=False, invert_xaxis=False, invert_yaxis=False, labelled=['x', 'y'], lod={'factor': 10, 'interval': 300, 'threshold': 2000, 'timeout': 500}, logx=False, logy=False, margin=(5, 10), max_height=None, max_width=None, min_height=None, min_width=None, multi_y=False, name='CurvePlot01696', normalize=True, padding=(0, 0.1), projection=None, responsive=False, scalebar=False, scalebar_label='@{value} @{unit}', scalebar_location='bottom_right', scalebar_opts={}, scalebar_range='x', scalebar_tool=True, scaleb

and finally renders the resulting plot output into the corresponding `Pane`:


In [150]:
print(pane)

Bokeh(figure, autodispatch=False, height=300, sizing_mode='fixed', width=300)
